In [1]:
# Prereq libraries
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import numpy as np
import scipy
import pandas as pb
import random
from nltk import word_tokenize

# Possibly useful TensorFlow and Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, CuDNNLSTM
from keras import layers
import tensorflow as tf

# Read in dataset from csv files
dataset = pb.read_csv('min_75_max5k_as_ids.csv')
# data = np.genfromtxt('min_75_max5k_as_ids.csv', delimiter = ',', usecols=np.arange(0,1000))
# data = [np.array(map(int, line.split())) for line in open('min_75_max5k_as_ids.csv')]
# data[1]

TRAINING_INPUT_FILE = "min75_max5k_as_ids.csv"

TRUE_TAGS_OUTPUT_FILE = "true_tags.csv"
PREDICTED_TAGS_OUTPUT_FILE = "predicted_tags.csv"
RANDOM_GAME_OUTPUT_FILE = "random_game_prediction.csv"

TRAINING_SPLIT = 0.2
ALPHA = 0.01

In [2]:
from functs import npFloatArray, load_data

# load features and labels based on those present in the CSV files
features = np.array(dataset.copy())

# Load Data
X_train, X_test, y_train, y_test = load_data(dataset, 
                                             TRAINING_SPLIT)

# Double check X_train's shape
print("X_train's Shape: " + str(X_train.shape[0]))

# Set up dataset for model fitting
num_rows, num_cols = X_train.shape
X_train = np.reshape(X_train, (X_train.shape[0],
                               1,
                               X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0],
                             1,
                             X_test.shape[1]))

# Test code used for printing results
# ========================================================
# all_tests= []
# all_results = []

# random_game_index = np.random.randint(0, len(all_X))
# random_game_description = all_X[random_game_index]
# random_game_results = []
# ========================================================

# Test code for shape sanity checks
# ========================================================
# print(num_cols)
# print(num_rows)
# print(y_train.shape[1:][0])

# print(random_game_index)
# print(random_game_description.shape)
# print(X_train.shape[1:])
# ========================================================

Max Y Index: 130
Max X Index: 3276
X_train's Shape: 14270


In [3]:
# ConvNN model implementation using Keras Library

# Create sequential model object
model = Sequential()

# First hidden layer using Convolution with window size 64 & RelU activation [3227 nodes]
model.add(Conv2D(num_cols, (3,3), input_shape =(X_train.shape[1:])))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Second  hidden layer using Convolution with RelU activation [3227 nodes]
model.add(Conv2D(num_cols,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Third hidden layer using fully conncted nodes for flattening
model.add(Flatten())
model.add(Dense(num_cols))

# Final output layer using softmax activation [131 nodes]
model.add(Dense(131))
model.add(Activation('sigmoid'))

# optimizer used for compilation, 'Adam' refers to gradient descent method that "adapts" using first and second order moments
opt = tf.keras.optimizers.Adam(learning_rate = 1e-3, decay = 1e-5)

# Final compilation
model.compile(loss = 'sparse_ctegorical_crossentropy', optimizer = opt, metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.RootMeanSquaredError()])

# Model Fitting process, this is the last step
# Note: Currently having issues with Tensor conversion causing this step to not accept training data
model.fit(X_train, y_train, batch_size = 3, validation_data = (X_test, y_test))

ValueError: Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 1, 3277)